In [4]:
from functools import cache
import requests


@cache
def get_prereqs(_subject: str, _course_number: str):
    return requests.get(
        "https://bulletin.temple.edu/ribbit/index.cgi",
        params={
            "page": "getcourse.rjs",
            "code": f"{_subject} {_course_number}",
        },
    ).text


In [ ]:
from bs4 import BeautifulSoup
import re

def parse_prereqs(_subject: str, _course_number: str):
    html = get_prereqs(_subject, _course_number)
    html = html[html.find("<![CDATA[") + 10 : html.rfind("]]>")]
    soup = BeautifulSoup(html, "html.parser")

    pre_reqs = ""

    for row in soup.select("p.courseblockpart"):
        text = row.get_text(strip=True)

        if text.find("Pre-requisites:") != -1:
            pre_reqs = text.replace("Pre-requisites:", "")
            break


    patterns = [
        "\\(",
        "\\)",
        "\\w+ \\d+",
        "(?<=in )[\\w\\d]+",
        "any .*? \\d+ to \\d+",
        "or",
        "and",
    ]

    pre_reqs = pre_reqs.replace("(may be taken concurrently)", "")
    pre_reqs = pre_reqs.replace("(C or higher)", "")
    pre_reqs = pre_reqs.replace("(except where noted)", "")

    m = re.findall("|".join(f"(?:{pattern})" for pattern in patterns), pre_reqs)

    final = ""
    depth = 0
    for str in m:



        match str:
            case "(":
                final += " " * 4 * depth
                final += "("
                depth += 1
            case ")":
                depth -= 1
                final += " " * 4 * depth
                final += ")"
            case "or" | "and":
                final += " " * 4 * depth
                final += str
            case _:
                final += " " * 4 * depth
                final += str



        final += "\n"

    print(final)
    print(pre_reqs)
    print()

parse_prereqs("CIS", "1068")

In [6]:
page = requests.get("https://bulletin.temple.edu/courses/cis/").text

In [8]:
from util.print import pprint

soup = BeautifulSoup(page, "html.parser")

str = []

for row in soup.select("p.courseblocktitle"):
    text = row.get_text(strip=True)
    text = text[:text.find(".")]

    # if text.find("Pre-requisites:") != -1:
    #     pre_reqs = text.replace("Pre-requisites:", "")
    #     break
    # print(text[0:4], text[4:8])
    str.append(text[4:8])

# pprint(a)

for b in str:
    parse_prereqs("CIS", b)




(
    any MATH course numbered 0701 to 0702
    any MATH course numbered 0800 to 0822
    any MATH course numbered 0824 to 0922
    any MATH course numbered 0924 to 4999
    MC3
    MC4
    MC5
    MC6
    STAT 1001
    STT2
    STAT 1102
    STAT 1902
    MC3A
    MC6A
    MATW
    MC3S
    MC3D
    MC3O
    MC3T
    or
    MC6T
)

Minimum grade of C- in (any MATH course numbered 0701 to 0702, any MATH course numbered 0800 to 0822 , any MATH course numbered 0824 to 0922 , any MATH course numbered 0924 to 4999 , 'Y' in MC3, 'Y' in MC4, 'Y' in MC5, 'Y' in MC6, STAT 1001 , 'Y' in STT2, STAT 1102 , STAT 1902 , 'Y' in MC3A, 'Y' in MC6A, 'Y' in MATW, 'Y' in MC3S, 'Y' in MC3D, 'Y' in MC3O, 'Y' in MC3T, or 'Y' in MC6T)




(
    any MATH course numbered 0701 to 0702
    any MATH course numbered 0800 to 0822
    any MATH course numbered 0824 to 0922
    any MATH course numbered 0924 to 4999
    MC3
    MC4
    MC5
    MC6
    STAT 1001
    STT2
    STAT 1102
    STAT 1902
    MC3A
    MC6A
